In [4]:
# RNN 클래스 구현
import numpy as np

class RNN:
    def __init__(self, Wx, Wh, b): # 입력값 가중치(Wx), 히든레이어 가중치(Wh), 편향(bias)
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np_zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
        
    # 순전파 구현
    def forward(self, x ,h_prev): # h_prev는 잘라낸 이전 계층의 출력값
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b # numpy의 matmul 메서드는 행렬곱을 수행한다.
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev, h_next) # 캐시에 이전 RNN 계층에서 받은 입력과 현 시각 RNN 계층으로부터의 출력을 저장(h_prev, h_next)
        return h_next
    
    # 역전파 구현
    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache
        
        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis=0)
        dWh = np.matmul(h_prev.T, dt)
        dh_prev = np.matmul(dt, Wh.T)
        dWx = np.matmul(x.T, dt)
        dx = np.matmul(dt, Wx.T)
        
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db
        
        return dx, dh_prev

In [6]:
# RNN 계층을 T개 연결한 Time RNN 신경망 구현
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None # 다수의 RNN 계층을 리스트로 저장하는 용도
        
        self.h, self.dh = None, None
        self.stateful = stateful
      
    # stateful을 긴 시계열 데이터를 처리할 때는 RNN의 은닉 상태 유지 여부를 boolean으로 받아서 은닉 상태를 유지한다.
    def set_state(self, h):
        self.h = h
    
    def reset_state(self):
        self.h = None
    
    # 순전파의 구현
    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape # N: 미니배치 크기, T: 시계열 데이터 분량, D: 벡터의 차원 수 xs는 T개 분량의 시계열 데이터를 하나로 모은 것
        # 따라서 xs의 형상은 (N, T, D)가 된다.
        D, H = Wx.shape
        
        self.layers = []
        hs = np.empty((N, T, H), dtype='f')
        
        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')
            
        for t in range(T):
            layer = RNN(*self.params) # * 연산자는 리스트의 원소들을 추출하여 메서드의 인수로 전달, 여기서는 RNN class의 __init__ 메서드의
            # 인수로 params 원소들을 전달한다. layer는 forward 내 존재하는 params 원소들을 인수로 전달한 RNN 객체가 됨.
            self.h = layer.forward(xs[:, t, :], self.h)
            hs[:, t, :] = self.h
            self.layers.append(layer)
            
        return hs # 시각을 리턴
    
    # 역전파 구현
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape
        
        dxs = np.empty((N, T, D), dtype='f')
        dh = 0
        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh) # 합산된 기울기
            dxs[:, t, :] = dx
            
            for i, grad in enumerate(layer.grads):
                grads[i] += grad
                
        
        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
            
        self.dh = dh
        
        return dxs
        